In [2]:
import pandas as pd
import json
import gzip
import ast

In [3]:
# Abrir y leer el archivo .json.gz

with gzip.open('steam_games.json.gz', 'rt', encoding='utf-8') as file:
    data = []
    for line in file:
        try:
            data.append(json.loads(line.strip()))  # Convertir de string a diccionario
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line}\nError: {e}")

# Crear un DataFrame de pandas
df = pd.DataFrame(data)

# Eliminar filas completamente vacías
df.dropna(how='all', inplace=True)

# Eliminar filas que tengan NaN en cualquier campo
df.dropna(inplace=True)

# Eliminar las columnas especificadas
columns_to_drop = ['publisher', 'app_name', 'url', 'reviews_url', 'specs','early_access']
df.drop(columns=columns_to_drop, inplace=True)


                                                   genres  \
88310       [Action, Casual, Indie, Simulation, Strategy]   
88311                [Free to Play, Indie, RPG, Strategy]   
88312   [Casual, Free to Play, Indie, Simulation, Sports]   
88313                         [Action, Adventure, Casual]   
88315                     [Action, Adventure, Simulation]   
...                                                   ...   
120439                 [Action, Adventure, Casual, Indie]   
120440              [Casual, Indie, Simulation, Strategy]   
120441                          [Casual, Indie, Strategy]   
120442                        [Indie, Racing, Simulation]   
120443                                    [Casual, Indie]   

                           title release_date  \
88310        Lost Summoner Kitty   2018-01-04   
88311                  Ironbound   2018-01-04   
88312    Real Pool 3D - Poolians   2017-07-24   
88313                    弹炸人2222   2017-12-07   
88315      Battle Roya

In [15]:
# Eliminar filas con fechas inválidas
df = df.dropna(subset=['release_date'])

# Eliminar filas duplicadas en 'id'
df = df.drop_duplicates(subset='id')

# Eliminar las filas con valores nulos en la columna 'release_date'
df = df.dropna(subset=['release_date'])

df = df.dropna(subset=['title'])

# Convertir la columna 'release_date' a formato datetime (si no lo es ya)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [11]:
# Asegúrate de que la columna sea de tipo string antes de hacer la comparación
df['price'] = df['price'].astype(str)

# Filtra los valores donde la longitud de los datos en 'price' sea menor o igual a 20 caracteres
df_filtrado = df[df['price'].str.len() <= 14]

# Si deseas modificar el DataFrame original, puedes reasignarlo
df = df_filtrado

# Convertimos la columna 'price' a numérica, y donde no pueda convertir, pone un 0
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)


# Si necesitas que 'price' sea entero:
df['price'] = df['price'].astype(float)

print(df)


# Muestra el DataFrame sin los datos que tenían más de 20 caracteres
print(df)


                                                   genres  \
88310       [Action, Casual, Indie, Simulation, Strategy]   
88311                [Free to Play, Indie, RPG, Strategy]   
88312   [Casual, Free to Play, Indie, Simulation, Sports]   
88313                         [Action, Adventure, Casual]   
88315                     [Action, Adventure, Simulation]   
...                                                   ...   
120439                 [Action, Adventure, Casual, Indie]   
120440              [Casual, Indie, Simulation, Strategy]   
120441                          [Casual, Indie, Strategy]   
120442                        [Indie, Racing, Simulation]   
120443                                    [Casual, Indie]   

                           title release_date  \
88310        Lost Summoner Kitty   2018-01-04   
88311                  Ironbound   2018-01-04   
88312    Real Pool 3D - Poolians   2017-07-24   
88313                    弹炸人2222   2017-12-07   
88315      Battle Roya

In [18]:
import mysql.connector
import json

# Conexión a la base de datos MySQL
conexion = mysql.connector.connect(
    host='35.226.92.249',
    port='3306',
    user='camilo',
    password='camilo_password',
    database='steam'
)

# Crear cursor
cursor = conexion.cursor()

# Crear la tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS games (
    id INT PRIMARY KEY,
    title VARCHAR(255),
    developer VARCHAR(255),
    release_date DATE,
    genres JSON,
    tags JSON,
    price VARCHAR(20)
);
"""
cursor.execute(create_table_query)

# Definir el tamaño del bloque
block_size = 1000  # Cambia esto según tus necesidades

# Insertar datos del DataFrame en la tabla en bloques
for start in range(0, len(df), block_size):
    end = start + block_size
    chunk = df[start:end]  # Obtén el bloque de datos

    # Crear la sentencia de inserción
    insert_query = """
    INSERT INTO games (id, title, developer, release_date, genres, tags, price)
    VALUES (%s, %s, %s, %s, %s, %s,%s)
    """
    
    # Ejecutar inserciones en bloque
    data_to_insert = []
    for index, row in chunk.iterrows():
        data_to_insert.append((
            row['id'],
            row['title'],
            row['developer'],
            row['release_date'],
            json.dumps(row['genres']),  # Convertir la lista a JSON
            json.dumps(row['tags']),     # Convertir la lista a JSON
            row['price']
        ))

    # Ejecutar la inserción de bloque
    cursor.executemany(insert_query, data_to_insert)

    # Confirmar los cambios de cada bloque
    conexion.commit()

# Cerrar la conexión
cursor.close()
conexion.close()

In [19]:
# Convertir datos a CSV file
df.to_csv('API/Datos/games.csv.gz', index=False, compression='gzip')